Flex all X-axes joints of the robox.

In [1]:
from matplotlib import pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
from os.path import dirname
sys.path.append(os.path.join(os.getcwd(), '..'))

In [3]:
import cv2
import pybullet as p
import numpy as np
import pylab

from opencv.opencv_tools import save
from pyb.pybullet_robot import PyBulletRobot
from opencv.opencv_pose import Pose
from phy.phy_robot import PhyRobot

In [4]:
vr = PyBulletRobot(render=True)

*** Initializing PyBulletRobot(ns=4, render=True) ...
*** Initializing PyBulletRobot() done


In [ ]:
rr = PhyRobot()

In [15]:
def rstep_1x(r, phix):
    phis = np.array([[phix, 0], [phix, 0], [phix, 0], [phix, 0]])
    r.step(phis)
    return phis

def vr_sweep():
    phiss, camps, tvecs, rvecs = [], [], [], []

    PHIX_LOW, PHIX_HIGH = np.pi/12, np.pi/6
    DPHI = (PHIX_HIGH - PHIX_LOW) / 25

    first_phi, last_phi = None, None
    vpose = Pose(vr.W, vr.H, '../pyb/cal.npz', vr.D)

    for phix in np.arange(PHIX_LOW, PHIX_HIGH+DPHI, DPHI):
        phis = rstep_1x(vr, phix)                  
        img, cam_p, _, _, _ = vr.getHeadcam()
        retval, rvec, tvec, _ = vpose.findChessboardRTVecs(img)
        if retval:
            camps.append(cam_p)
            tvecs.append(tvec)
            rvecs.append(rvec)
            phiss.append(phis)

            if first_phi is None:
                first_phi = last_phi = phix
                print("vr first_phi: %f" % first_phi)
            elif last_phi < phix:
                last_phi = phix

    print("vr last_phi: %f" % last_phi)


    phiss = np.array(phiss)
    camps = np.array(camps)
    tvecs = np.array(tvecs)
    rvecs = np.array(rvecs)

    return phiss, camps, tvecs, rvecs

def rr_sweep():
    phiss, camps, tvecs, rvecs = [], [], [], []

    PHIX_LOW, PHIX_HIGH = np.pi/12, np.pi/6
    DPHI = (PHIX_HIGH - PHIX_LOW) / 25

    first_phi, last_phi = None, None
    vpose = Pose(vr.W, vr.H, '../phy/cal.npz', vr.D)

    for phix in np.arange(PHIX_LOW, PHIX_HIGH+DPHI, DPHI):
        phis = rstep_1x(rr, phix)                  
        img, cam_p, _, _, _ = vr.getHeadcam()
        retval, rvec, tvec, _ = vpose.findChessboardRTVecs(img)
        if retval:
            camps.append(cam_p)
            tvecs.append(tvec)
            rvecs.append(rvec)
            phiss.append(phis)

            if first_phi is None:
                first_phi = last_phi = phix
                print("rr first_phi: %f" % first_phi)
            elif last_phi < phix:
                last_phi = phix

    print("rr last_phi: %f" % last_phi)


    phiss = np.array(phiss)
    camps = np.array(camps)
    tvecs = np.array(tvecs)
    rvecs = np.array(rvecs)

    return phiss, camps, tvecs, rvecs

## VR robot

In [16]:
phiss, camps, tvecs, _ = vr_sweep()

first_phi: 0.324631
last_phi: 0.439823


#### head moves down on Z world axis

In [ ]:
plt.plot(camps[:,2], 'g.')

#### target Y component of translation vector sweeps from 0.6 to -0.6

In [ ]:
plt.plot(camps[:,2], 'g.', tvecs[:,1], 'r.')

#### headcam sways horizontally a tiny bit

In [ ]:
plt.plot(camps[:,1], 'g.', tvecs[:,0], 'r.')

#### headcam gets closer to the target

In [ ]:
camds = []
for cam_p in camps:
    cam_d = np.sqrt(np.sum(np.square(np.array(cam_p) - np.array([2, 0, 0.59])), axis=0))
    camds.append(cam_d)
camds = np.array(camds)

In [ ]:
tvecds = np.sqrt(np.sum(np.square(tvecs), axis=1)).ravel()

In [ ]:
plt.plot(camds, 'g', tvecds, 'r')

In [ ]:
tvecds

In [ ]:
plt.plot(camds - tvecds, 'b')

####  Z component of translation vector reaches it peak and goes away

In [ ]:
plt.plot(tvecs[:,2], 'r.')

In [ ]:
r.close()